In [1]:
import fxcmpy
TOKEN = "65aa373e2c6d310060a655ac665b46531ecd1adc"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

In [23]:
import datetime as dt
start = dt.datetime(2021,12,23)
end = dt.datetime(2021,12,24)
data = con.get_candles('EUR/USD', period='m1', number=250)
data

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2021-12-27 02:27:00,1.13170,1.13176,1.13179,1.13170,1.13184,1.13188,1.13192,1.13183,42
2021-12-27 02:28:00,1.13176,1.13175,1.13180,1.13175,1.13189,1.13188,1.13194,1.13188,41
2021-12-27 02:29:00,1.13174,1.13175,1.13175,1.13173,1.13187,1.13187,1.13187,1.13186,63
2021-12-27 02:30:00,1.13174,1.13185,1.13185,1.13173,1.13187,1.13198,1.13198,1.13186,63
2021-12-27 02:31:00,1.13185,1.13184,1.13186,1.13183,1.13199,1.13198,1.13199,1.13197,19
...,...,...,...,...,...,...,...,...,...
2021-12-27 06:32:00,1.13145,1.13149,1.13152,1.13144,1.13157,1.13162,1.13164,1.13157,25
2021-12-27 06:33:00,1.13149,1.13158,1.13159,1.13148,1.13161,1.13170,1.13172,1.13159,56
2021-12-27 06:34:00,1.13159,1.13163,1.13164,1.13157,1.13171,1.13176,1.13176,1.13169,91


In [3]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])

In [19]:
import ta
from ta.trend import *
from ta.momentum import *


data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# Stochastic need to fix
data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)

isinstance(data,pd.DataFrame)

True

In [29]:
class BaseStrategy:
    name = "base_strategy"
    description = ""
    df_validated = False
    validation_initiated = False
    

    def __init__(self, *args, **kwargs) -> None:
        self.__dataframe = None
        if self.df_validated or self.validation_initiated:
            raise AttributeError("df_validated, or validation_initiated can not be true by default")
        if not self.name:
            raise AttributeError("strategy name is mandatory")
    
    @property
    def dataframe(self):
        return self.__dataframe
    
    @dataframe.setter
    def dataframe(self, dataframe):
        if not isinstance(dataframe, pd.DataFrame):
                raise TypeError("not valid dataframe")
        self.__dataframe = dataframe
        
    
    def __strategy(self):
        pass
    
    @classmethod
    def dataframe_validation_done(cls):
        cls.validation_initiated = True
        cls.df_validated = True
    
    def is_valid_dataframe(self):
        if self.dataframe is None:
            raise AttributeError("dataframe value must be passed")
        if len(self.dataframe.index) > 50:
            self.__class__.dataframe_validation_done()
            return True
        print("dataframe should have minimum 50 rows")
        return False
    
    def get_signal(self):
        if not all([self.df_validated, self.validation_initiated]):
            print("please validate dataframe first by calling validate_dataframe function")
            return None
        return self.__strategy()
        
    
    
c = BaseStrategy()
c.dataframe = data.head(20)
c.is_valid_dataframe()
c.df_validated

dataframe should have minimum 50 rows


False